# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**. 

---

<img src="../img/reflection_pattern.png" alt="Alt text" width="600"/>

---

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

## Generation Step

The first thing we need to consider is:

> What do we want to generate? A poem? An essay? Python code?

For this example, I've decided to test the Python coding skills of Llama3 70B (that's the LLM we are going to use for all the tutorials). In particular, we are going to ask our LLM to code a famous sorting algorithm: **Merge Sort**. 

---

<img src="../img/mergesort.png" alt="Alt text" width="500"/>

### Groq Client and relevant imports

In [1]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

client = Groq()

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [2]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [3]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm"
    }
)

Let's generate the first version of the essay.

In [4]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [5]:
display_markdown(mergesort_code, raw=True)

Here is a Python implementation of the Merge Sort algorithm:
```
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array to be sorted
    :return: The sorted array
    """
    # Base case: If the array has only one element, it is already sorted
    if len(arr) <= 1:
        return arr

    # Split the array into two halves
    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    # Recursively sort the two halves
    left = merge_sort(left)
    right = merge_sort(right)

    # Merge the two sorted halves
    return merge(left, right)

def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result.extend(left)
    result.extend(right)
    return result
```
Here's an explanation of how the code works:

1. The `merge_sort` function takes an array as input and recursively splits it into two halves until each half has only one element.
2. The `merge` function takes two sorted arrays as input and merges them into a single sorted array.
3. The `merge` function uses a temporary array `result` to store the merged elements.
4. The `merge` function compares elements from both arrays and adds the smaller one to the `result` array.
5. Once one of the input arrays is empty, the remaining elements from the other array are appended to the `result` array.
6. The `merge_sort` function returns the sorted array.

Example usage:
```
arr = [5, 2, 8, 3, 1, 6, 4]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```
Let me know if you have any feedback or if you'd like me to revise anything!

## Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. This system prompt will tell the LLM to act as Andrej Karpathy, computer scientist and Deep Learning wizard.

>To be honest, I don't think the fact of acting like Andrej Karpathy will influence the LLM outputs, but it was fun :)

<img src="../img/karpathy.png" alt="Alt text" width="500"/>

In [6]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [7]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

Now, let's generate a critique to the Python code.

In [8]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [9]:
display_markdown(critique, raw=True)

Great implementation of Merge Sort! Here's my feedback:

**Overall structure and organization**: Your code is well-organized, and the separation of the `merge_sort` and `merge` functions is correct. The docstrings are also helpful in explaining the purpose and behavior of each function.

**Code quality and style**: Your code adheres to PEP 8 conventions, which is great. The variable names are descriptive, and the code is easy to read.

**Performance**: Your implementation has the correct time and space complexity for Merge Sort, which is O(n log n) and O(n), respectively.

**Minor suggestions**:

1. **Consistent spacing**: In the `merge` function, there's an extra space between the `while` loop and the `result.extend` statements. Remove the extra space for consistency.
2. **In-line comments**: You could add in-line comments to explain the logic behind specific lines of code, especially in the `merge` function. For example, you could add a comment above the `while` loop explaining that it's merging the two sorted arrays.
3. **Type hints**: Consider adding type hints for the function parameters and return types to make the code more readable and self-documenting.
4. **Testing**: While you provided an example usage, it would be great to include more test cases to ensure the implementation is correct. You could add a test function that verifies the sorted array is correct for different input cases.

Here's the revised code with the minor suggestions incorporated:
```python
def merge_sort(arr: list[int]) -> list[int]:
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array to be sorted
    :return: The sorted array
    """
    # Base case: If the array has only one element, it is already sorted
    if len(arr) <= 1:
        return arr

    # Split the array into two halves
    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    # Recursively sort the two halves
    left = merge_sort(left)
    right = merge_sort(right)

    # Merge the two sorted halves
    return merge(left, right)

def merge(left: list[int], right: list[int]) -> list[int]:
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    # Merge the two sorted arrays
    while len(left) > 0 and len(right) > 0:
        # Compare elements from both arrays and add the smaller one to the result
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    # Append the remaining elements from the non-empty array
    result.extend(left)
    result.extend(right)
    return result

# Example usage
arr = [5, 2, 8, 3, 1, 6, 4]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```
Overall, your implementation is correct and well-structured. With these minor suggestions, it becomes even more readable and maintainable.

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [10]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

## Generation Step (II)

In [11]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [12]:
display_markdown(essay, raw=True)

Thank you for the thorough feedback! I'm glad to hear that the overall structure and organization of the code is good, and that the code quality and style adhere to PEP 8 conventions.

I appreciate the minor suggestions you provided, and I agree that they can improve the code. Here's a revised version of the code that incorporates your suggestions:
```
def merge_sort(arr: list[int]) -> list[int]:
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array to be sorted
    :return: The sorted array
    """
    # Base case: If the array has only one element, it is already sorted
    if len(arr) <= 1:
        return arr

    # Split the array into two halves
    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    # Recursively sort the two halves
    left = merge_sort(left)
    right = merge_sort(right)

    # Merge the two sorted halves
    return merge(left, right)

def merge(left: list[int], right: list[int]) -> list[int]:
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    # Merge the two sorted arrays
    while len(left) > 0 and len(right) > 0:
        # Compare elements from both arrays and add the smaller one to the result
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    # Append the remaining elements from the non-empty array
    result.extend(left)
    result.extend(right)
    return result

def test_merge_sort():
    # Test case 1: Already sorted array
    arr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    assert merge_sort(arr) == arr

    # Test case 2: Reverse sorted array
    arr = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
    assert merge_sort(arr) == [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

    # Test case 3: Unsorted array
    arr = [5, 2, 8, 3, 1, 6, 4]
    assert merge_sort(arr) == [1, 2, 3, 4, 5, 6, 8]

    # Test case 4: Array with duplicates
    arr = [5, 2, 8, 3, 1, 6, 4, 3, 5, 2]
    assert merge_sort(arr) == [1, 2, 2, 3, 3, 4, 5, 5, 6, 8]

    print("All test cases pass!")

test_merge_sort()
```
I've added type hints for the function parameters and return types, as well as in-line comments to explain the logic behind specific lines of code. I've also added a `test_merge_sort` function that includes four test cases to verify the correctness of the implementation.

Thank you again for the feedback! I'm glad to have the opportunity to revise and improve the code.

## And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

## Implementing a class 

Now that you understand the underlying loop of the Reflection Agent, let's implement this agent as a class.

In [13]:
from agentic_patterns import ReflectionAgent

ModuleNotFoundError: No module named 'agentic_patterns'

In [14]:
agent = ReflectionAgent()

In [15]:
generation_system_prompt = "You are a Python programmer tasked with generating high quality Python code"

reflection_system_prompt = "You are Andrej Karpathy, an experienced computer scientist"

user_msg = "Generate a Python implementation of the Merge Sort algorithm"

In [ ]:
final_response = agent.run(
    user_msg=user_msg,
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    n_steps=3,
    verbose=1,
)

## Final result

In [ ]:
display_markdown(final_response, raw=True)